# Delta branch

In [2]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri

def ax_plot_simplex(ax, optimal_vertex, sim, jdrop, rho):
    simplex = np.array((optimal_vertex, *(sim + optimal_vertex), optimal_vertex))
    ax.plot(simplex[...,0], simplex[...,1], '-', color='lightsteelblue', lw=2)
    ax.plot(simplex[1:-1,0], simplex[1:-1,1], '.', color='blue', marker='o')
    ax.plot(simplex[jdrop+1,0], simplex[jdrop+1,1], '.', color='red', marker='o')
    ax.plot(*optimal_vertex, color='limegreen', marker='D')
    
    beta = 2.1
    beta_region = plt.Circle(optimal_vertex, beta * rho, color='lemonchiffon', fill=True, alpha=0.5)
    trust_region = plt.Circle(optimal_vertex, rho, color='gold', fill=True, alpha=0.5)
    ax.add_patch(beta_region)
    ax.add_patch(trust_region)

In [3]:
# eta case
plt.close('all')
jdrop = 0
rho = 0.0625
optimal_vertex = np.array([0.047873700624370373047, 0.03301727499567987112])
old_sim = np.array([[-0.13019926157022278929, -0.059595187823573939068], [-0.05055567780160368646, 0.036747427692566961288]])
new_sim = np.array([[0.018373713846283478442, 0.025277838900801840201], [-0.05055567780160368646, 0.036747427692566961288]])
old_veta = np.array([0.14319020261581568261, 0.062500000000000007495])
new_veta = np.array([0.03125, 0.062500000000000007495])
old_vsig = np.array([0.12475780902899252224, 0.054454585033534948986])
new_vsig = np.array([0.031249999999999999998, 0.06250000000000000749])

coords = np.array((*old_sim, *new_sim)) + optimal_vertex
x_coords = coords[:, 0]
y_coords = coords[:, 1]

plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
xmin = min(x_coords)
xmax = max(x_coords)
ymin = min(y_coords)
ymax = max(y_coords)
margin = 1e-2

#_ = ax1.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
#_ = ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
ax1.set_aspect('equal')
ax2.set_aspect('equal')

ax_plot_simplex(ax1, optimal_vertex, old_sim, jdrop, rho=rho)
ax_plot_simplex(ax2, optimal_vertex, new_sim, jdrop, rho=rho)
plt.savefig('mem/rsc/delta-eta.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# sigma case
plt.close('all')
jdrop = 1
rho = 6.103515625e-05
optimal_vertex = np.array([0.12322095602324728612, 0.7876477600799215061])
old_sim = np.array([[0.00010609379901699572403, -5.758107655831259315e-05], [4.8716293255678108707e-05, -3.676973034832708352e-05]])
#new_sim = np.array([[0.00010609379901699572403, -5.758107655831259315e-05], [-1.4557208222393858973e-05, -2.6821824385849429767e-05]])
new_sim = np.array([[0.000120651007239389583, -3.0759252172463163382e-05], [1.4557208222393858973e-05, 2.6821824385849429767e-05]])
old_veta = np.array([0.00012071236294378856573, 6.103515625000000053e-05])
#new_veta = np.array([0.00012071236294378856573, 3.0517578125e-05])
new_veta = np.array([0.00012451022906608282922, 3.0517578125e-05])
old_vsig = np.array([1.7955287365912709351e-05, 9.078637375381810534e-06])
#new_vsig = np.array([0.00012071236294378856568, 3.0517578124999999998e-05])
new_vsig = np.array([0.00012071236294378856568, 2.958671744821261666e-05])

coords = np.array((*old_sim, *new_sim)) + optimal_vertex
x_coords = coords[:, 0]
y_coords = coords[:, 1]

plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
xmin = min(x_coords)
xmax = max(x_coords)
ymin = min(y_coords)
ymax = max(y_coords)
margin = 6e-5

#_ = ax1.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
#_ = ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
ax1.set_aspect('equal')
ax2.set_aspect('equal')

ax_plot_simplex(ax1, optimal_vertex, old_sim, jdrop, rho=rho)
ax_plot_simplex(ax2, optimal_vertex, new_sim, jdrop, rho=rho)
plt.savefig('mem/rsc/delta-sigma.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Examples

In [78]:
import functools
import itertools

import numpy as np

from pycobyla.gcobyla import GCobyla

import tests.test_custom as tc
from tests.nb import utils

In [79]:
%matplotlib widget

import matplotlib.pyplot as plt
from matplotlib import cm

def ax_plot_simplex(ax, optimal_vertex, sim):
    simplex = np.array((optimal_vertex, *(sim + optimal_vertex), optimal_vertex))
    ax.plot(*optimal_vertex, color='limegreen', marker='o')
    ax.plot(simplex[1:-1,0], simplex[1:-1,1], '.', color='blue', marker='o')
    ax.plot(simplex[...,0], simplex[...,1], '-.', color='gray', lw=2)
    

def ax_plot_track(opt, ax, target, from_points=0, to_points=None, plot_simplex=False, plot_trure=False, aspect=False, track_color=None, track_alpha=1):
    track = opt.track[from_points:to_points]
    ax.plot(track[:, 0], track[:, 1], linestyle='-', color=track_color, alpha=track_alpha, marker='o')

    link_best = np.array((track[-1], target))
    ax.plot(link_best[:, 0], link_best[:, 1], linestyle=':', color=track_color)
    ax.plot(target[0], target[1], color='limegreen', marker='*')
        
    if plot_simplex:
        #simplex = np.array((opt.optimal_vertex, *(opt.sim + opt.optimal_vertex), opt.optimal_vertex))
        ax_plot_simplex(ax, opt.optimal_vertex, opt.sim)
            
    if plot_trure:
        trust_region = plt.Circle(opt.optimal_vertex, opt.rho, color='khaki', fill=True, alpha=0.5)
        ax.add_patch(trust_region)
        
    if aspect:
        ax.set_aspect('equal')
        
def ax_plot_map(fig, ax, opt, border, nsamples=100):
    X = np.linspace(-border, border, nsamples)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(-opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)
    
    cs = ax.contourf(xv, yv, zv, cmap=cm.magma)
    #cbar = fig.colorbar(cs)
    
    #ax.plot(np.array((-border, border)), np.array((0, 0)), linestyle='-.', color='royalblue')
    #ax.plot(np.array((0, 0)), np.array((-border, border)), linestyle='-.', color='royalblue')


In [80]:
def gaussian_optimizer(start_x, mu=None, sig=None, A=-1):
    mu = np.zeros(2) if mu is None else mu
    sig = np.ones(2) if sig is None else sig
    
    F = functools.partial(tc.gaussian, mu=mu, sig=sig, A=A)
    C = ()
    
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [81]:
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 9))
mu = (0, 0)

gg = functools.partial(gaussian_optimizer, mu=mu, A=-1)

rsc = {
    axes[0,0]: ((.5**.5,.5**.5),'a)'),
    axes[0,1]: ((0,1), 'b)'),
    axes[0,2]: ((-.5**.5,.5**.5), 'c)'),
    axes[1,0]: ((-.5**.5,-.5**.5), 'd)'),
    axes[1,1]: ((0,-1), 'e)'),
    axes[1,2]: ((.5**.5,-.5**.5),'f)'),
}
    

for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x)
    for _ in opt.g_run(): pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f})')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=50)
    
plt.savefig('mem/rsc/ex1-1.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Una normal amb restriccions

In [82]:
def gaussian_optimizer(start_x, border=1, mu=None, sig=None, A=-1):
    mu = np.zeros(2) if mu is None else mu
    sig = np.ones(2) if sig is None else sig
    
    F = functools.partial(tc.gaussian, mu=mu, sig=sig, A=A)
    c1 = lambda x: border - x[0]
    c2 = lambda x: border + x[0]
    c3 = lambda x: border - x[1]
    c4 = lambda x: border + x[1]
    C = (c1, c2, c3, c4)
    
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [83]:
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 9))
border = .5
mu = (0, 0)

gg = functools.partial(gaussian_optimizer, border=border, mu=mu, A=-1)

rsc = {
    axes[0,0]: ((.5**.5,.5**.5),'a)'),
    axes[0,1]: ((0,1), 'b)'),
    axes[0,2]: ((-.5**.5,.5**.5), 'c)'),
    axes[1,0]: ((-.5**.5,-.5**.5), 'd)'),
    axes[1,1]: ((0,-1), 'e)'),
    axes[1,2]: ((.5**.5,-.5**.5),'f)'),
}
    

for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x)
    for _ in opt.g_run(): pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f})')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=50)
    
    region = np.array(((border, border), (-border,border), (-border, -border), (border, -border), (border, border)))
    ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='black')
    ax.fill(region[..., 0], region[..., 1], color='limegreen', alpha=.35)
    
plt.savefig('mem/rsc/ex1-2.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 9))
mu = (0, 0)
border = 1

gg = functools.partial(gaussian_optimizer, mu=mu, A=-1)

start_x = (.5**.5, .5**.5)
rsc = {
    axes[0,0]: (start_x, 'a)'),
    axes[0,1]: (start_x, 'b)'),
    axes[0,2]: (start_x, 'c)'),
    axes[1,0]: (start_x, 'd)'),
    axes[1,1]: (start_x, 'e)'),
    axes[1,2]: (start_x, 'f)'),
}
    

for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x, border=border)
    for _ in opt.g_run(): pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f}), length side {2*border:.2f}')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=50)
    
    region = np.array(((border, border), (-border,border), (-border, -border), (border, -border), (border, border)))
    ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='black')
    ax.fill(region[..., 0], region[..., 1], color='limegreen', alpha=.35)
    border *= .5
plt.savefig('mem/rsc/ex1-3.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Normal amb regió delimitada per dues rectes. x inicial fora de la regió especial

In [85]:
def gaussian_optimizer(start_x, mu=None, sig=None, A=-1):
    mu = np.zeros(2) if mu is None else mu
    sig = np.ones(2) if sig is None else sig
    
    F = functools.partial(tc.gaussian, mu=mu, sig=sig, A=A)
    c1 = lambda x: x[0] - x[1] - .5
    c2 = lambda x: -(x[0] - x[1] - 1)
    C = (c1, c2)
    
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [86]:
plt.close('all')
fig, axes = plt.subplots(1,2, figsize=(9, 4))
mu = (0, 0)

gg = functools.partial(gaussian_optimizer, mu=mu, A=-1)

rsc = {
    axes[0]: ((-1, 0), 'a)'),
    axes[1]: ((1.8, 0), 'b)'),
}
    
for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x)
    for _ in opt.g_run(): pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f})')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=100)
    
    l1 = lambda x: x - .5
    l2 = lambda x: x - 1
    c1 = np.array(((-1.5, l1(-1.5)), (2, l1(2))))
    c2 = np.array(((-1, l2(-1)), (2, l2(2))))
    ax.plot(c1[..., 0], c1[..., 1], linestyle='-.', color='black')
    ax.plot(c2[..., 0], c2[..., 1], linestyle='-.', color='black')
    
    region = np.array(((-1.5, l1(-1.5)), (2, l1(2)), (2, l2(2)), (-1, l2(-1)), (-1.5, l1(-1.5))))
    ax.fill(region[..., 0], region[..., 1], color='limegreen', alpha=.35)
    print(opt.optimal_vertex)

plt.savefig('mem/rsc/ex1-4.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.25 -0.25]
[ 0.25 -0.25]


In [87]:
import matplotlib.cm as cm

plt.close('all')
fig, ax = plt.subplots(1, figsize=(9, 4))
mu = (0, 0)

ax1.set_aspect('equal')
ax2.set_aspect('equal')

gg = functools.partial(gaussian_optimizer, mu=mu, A=-1)
angles = np.arange(start=0, stop=2*np.pi, step=15*(2*np.pi)/360)

for angle in angles:
    start_x = np.array((np.cos(angle), np.sin(angle)))
    opt = gg(start_x=start_x)
    for _ in opt.g_run(): pass
    
    ax_plot_track(opt, ax, mu, aspect=True, track_color='darkred',  track_alpha=.3)
    ax.plot(start_x[0], start_x[1], 'o', color='darkred')
    ax.plot(opt.optimal_vertex[0], opt.optimal_vertex[1], 'o', color='orange')
    
    l1 = lambda x: x - .5
    l2 = lambda x: x - 1
    c1 = np.array(((-1.5, l1(-1.5)), (2, l1(2))))
    c2 = np.array(((-1, l2(-1)), (2, l2(2))))
    

    assert (opt.optimal_vertex.round(9) == np.array((.25, -.25))).all()
    #print(opt.optimal_vertex)
        
ax_plot_map(fig, ax, opt, border=2, nsamples=100)
ax.plot(c1[..., 0], c1[..., 1], linestyle='-.', color='black')
ax.plot(c2[..., 0], c2[..., 1], linestyle='-.', color='black')
region = np.array(((-1.5, l1(-1.5)), (2, l1(2)), (2, l2(2)), (-1, l2(-1)), (-1.5, l1(-1.5))))
ax.fill(region[..., 0], region[..., 1], color='limegreen', alpha=.35)

plt.savefig('mem/rsc/ex1-5.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Out of circle

In [88]:
def optimization_out_of_the_boundary_circle(inner_radius=1, outer_radius=2, start_x=(0,0), mu=None):
    assert outer_radius > inner_radius
    
    sig = np.random.choice((-1, 1))
    phi = np.random.uniform(low=0, high=2*np.pi*outer_radius) / outer_radius
    x_circle = outer_radius * np.cos(phi)
    y_circle = outer_radius * np.sin(phi)
    
    mu = np.array((x_circle, y_circle)) if mu is None else mu
    
    G = functools.partial(tc.gaussian, mu=mu, A=-1)
    c1 = lambda x: inner_radius - sum(x ** 2)

    C = (c1,)

    opt = GCobyla(start_x, G, C, rhobeg=3e-1, rhoend=1e-15)
    for _ in opt.g_run(): pass
    
    
    return mu, opt

In [89]:
%matplotlib widget
import matplotlib.pyplot as plt
plt.close('all')


inner_radius = 1
outer_radius = 2
start_x = np.array((-0.10696211, -0.65293934))
#start_x = np.random.uniform(low=-1, high=1, size=2)
mu = np.array([0.17174320179775046, 1.9926124240896064])
#mu = None

mu, opt = optimization_out_of_the_boundary_circle(
    inner_radius=inner_radius, outer_radius=outer_radius, start_x=start_x, mu=mu)
print(mu.tolist())

plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
ax1.axis('equal')

inner_circle = plt.Circle((0, 0), inner_radius, color='limegreen', fill=True, alpha=.35)  # boundary
outer_circle = plt.Circle((0, 0), outer_radius, color='lightgrey', fill=False)
ax1.add_patch(inner_circle)
ax1.add_patch(outer_circle)

ax_plot_track(opt, ax1, mu,  aspect=True, track_color='darkred')
ax1.plot(*start_x, marker='o', color='blue')
ax1.plot(*opt.x, marker='o', color='orange')
#ax1.plot(*mu, marker='*', color='limegreen')
#ax1.plot((opt.x[0], mu[0]), (opt.x[1], mu[1]), linestyle='-.', color='darkred')

ax1.set_xlabel(f'a) mu at ({mu[0]:.2f}, {mu[1]:.2f})')

n_targets = 25
data = tuple(optimization_out_of_the_boundary_circle(inner_radius=inner_radius, outer_radius=outer_radius, start_x=start_x) 
             for _ in range(n_targets))

v_mu = np.array(tuple(mu for mu, opt in data))
v_opt = np.array(tuple(opt.x for mu, opt in data))


ax2.axis('equal')

inner_circle = plt.Circle((0, 0), inner_radius, color='limegreen', fill=True, alpha=.35)  # boundary
outer_circle = plt.Circle((0, 0), outer_radius, color='lightgrey', fill=False)
ax2.add_patch(inner_circle)
ax2.add_patch(outer_circle)

ax2.plot(*start_x, marker='o', color='blue')
ax2.plot(v_opt[:, 0], v_opt[:, 1], linestyle='', marker='o', color='orange')
ax2.plot(v_mu[:, 0], v_mu[:, 1],  linestyle='', marker='*', color='limegreen')
for mu, opt_x in zip(v_mu, v_opt):
    ax2.plot((opt_x[0], mu[0]), (opt_x[1], mu[1]), linestyle=':', color='darkred')

ax2.set_xlabel(f'b) {n_targets} independent opt.')    
plt.savefig('mem/rsc/ex1-6.png', dpi=120, bbox_inches='tight')

[0.17174320179775046, 1.9926124240896064]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
plt.close('all')
from matplotlib.patches import Rectangle

def optimization_out_of_the_boundary_square(side=1, outer_radius=2, start_x=(0,0), mu=None):
    phi = np.random.uniform(low=0, high=2*np.pi*outer_radius) / outer_radius
    x_circle = outer_radius * np.cos(phi)
    y_circle = outer_radius * np.sin(phi)
    
    mu = np.array((x_circle, y_circle)) if mu is None  else mu
    
    G = functools.partial(tc.gaussian, mu=mu, A=-1)
    c1 = lambda x: side/2 - x[0]
    c2 = lambda x: side/2 + x[0]
    c3 = lambda x: side/2 - x[1]
    c4 = lambda x: side/2 + x[1]

    C = (c1, c2, c3, c4)

    opt = GCobyla(start_x, G, C, rhoend=1e-12)
    for _ in opt.g_run(): pass
    return mu, opt

#####
side = 2
outer_radius = 2
start_x = np.array([-1.269592892310348, -1.191539685399302])
#start_x = np.random.uniform(low=-1.5, high=1.5, size=2)
mu = np.array([1.7503076742188297, 0.9676895398683771])
#mu = None

mu, opt = optimization_out_of_the_boundary_square(side=side, outer_radius=outer_radius, start_x=start_x, mu=mu)

plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
ax1.axis('equal')

boundary = plt.Rectangle((-side/2, -side/2), side, side, color='limegreen', fill=True, alpha=.35)
outer_circle = plt.Circle((0, 0), outer_radius, color='lightgrey', fill=False)
ax1.add_patch(boundary)
ax1.add_patch(outer_circle)

ax_plot_track(opt, ax1, mu,  aspect=True, track_color='darkred')
ax1.plot(*start_x, marker='o', color='blue')
ax1.plot(*opt.x, marker='o', color='orange')
ax1.plot(*mu, marker='*', color='limegreen')
#ax.plot((start_x[0], opt.x[0]), (start_x[1], opt.x[1]), linestyle='-', color='red')
ax1.plot((opt.optimal_vertex[0], mu[0]), (opt.optimal_vertex[1], mu[1]), linestyle=':', color='darkred')
ax1.set_xlabel(f'a) mu at ({mu[0]:.2f}, {mu[1]:.2f})')


n_targets = 25
data = tuple(optimization_out_of_the_boundary_square(side=side, outer_radius=outer_radius, start_x=start_x) 
             for _ in range(n_targets))
v_mu = np.array(tuple(mu for mu, opt in data))
v_opt = np.array(tuple(opt.x for mu, opt in data))

ax2.axis('equal')
boundary = plt.Rectangle((-side/2, -side/2), side, side, color='limegreen', fill=True, alpha=.35)
outer_circle = plt.Circle((0, 0), outer_radius, color='lightgrey', fill=False)
ax2.add_patch(boundary)
ax2.add_patch(outer_circle)

ax2.plot(*start_x, marker='o', color='blue')
ax2.plot(v_opt[:, 0], v_opt[:, 1], linestyle='', marker='o', color='orange')
ax2.plot(v_mu[:, 0], v_mu[:, 1],  linestyle='', marker='*', color='limegreen')

for mu, opt_x in zip(v_mu, v_opt):
    #ax.plot((start_x[0], opt_x[0]), (start_x[1], opt_x[1]), linestyle='-', color='red')
    ax2.plot((opt_x[0], mu[0]), (opt_x[1], mu[1]), linestyle=':', color='darkred')
    
ax2.set_xlabel(f'b) {n_targets} independent opt.') 
plt.savefig('mem/rsc/ex1-7.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Pyramid Example

In [91]:
import functools

import tests.test_custom as tc

radius = 2

def pyramid_faces_optimizer(start_x, radius=radius, faces=4, rhobeg=.5, rhoend=1e-12):
    F = functools.partial(tc.pyramid_faces, center=np.zeros(2), radius=radius, height=-1, faces=faces)
    c1 = lambda x: 1 - sum(x ** 2)
    C = (c1,)
    opt = GCobyla(start_x, F, C, rhobeg=rhobeg, rhoend=rhoend)
    return opt

In [92]:
%matplotlib widget
#%matplotlib notebook
import itertools

import numpy as np

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')

X = np.linspace(-radius, radius, 200)
xv, yv = np.meshgrid(X, X)
zv = np.array(tuple(tc.pyramid_faces(xy, center=(0, 0), radius=radius, height=1, faces=4) for xy in itertools.product(X, X)))
zv = zv.reshape(xv.shape)

# Plot the surface.
surf = ax.plot_surface(xv, yv, zv, cmap=cm.inferno, linewidth=0, antialiased=False)

# Customize the z axis.
ax.zaxis.set_major_locator(LinearLocator(5))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.01f'))

plt.savefig('mem/rsc/ex1-8.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
#start_x = np.array((-0.26835858024991177117613, -0.12417347476622397905999))  #radius = 1
#start_x = np.array((0.29437038734662568373324, -0.55939322414647030257129))  # radius = 2
#start_x = np.array((0.60522848317453536992616, -0.92243950619371162247262))  # radius = 2
start_x = np.array((0.6052, -0.9224))  # radius = 2
opt = pyramid_faces_optimizer(start_x=start_x, faces=4, rhobeg=.5)
for _ in opt.g_run(): pass

plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(10, 9))
ax1 = axes[0,0]
ax2 = axes[0,1]
ax3 = axes[1,0]
ax4 = axes[1,1]

circle = plt.Circle((0, 0), 1, color='limegreen', fill=False)

###
#ax1.set_aspect('equal')
ax1.axis('equal')

ax1.plot(np.array((-4, 4)), np.array((0, 0)), linestyle='-.', color='black')
ax1.plot(np.array((0, 0)), np.array((-4, 4)), linestyle='-.', color='black')

ax_plot_track(opt, ax1, target=np.zeros(2),  aspect=True, track_color='darkred')
ax_plot_map(fig, ax1, opt, border=2, nsamples=200)
ax1.add_patch(circle)
ax1.set(xlim=(-1, 1), ylim=(-1, 1))
ax1.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax1.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

###
#ax2.set_aspect('equal')
ax2.axis('equal')
ax2.plot(np.array((-4, 4)), np.array((0, 0)), linestyle='-.', color='black')
ax2.plot(np.array((0, 0)), np.array((-4, 4)), linestyle='-.', color='black')

ax_plot_track(opt, ax2, target=np.zeros(2),  aspect=True, track_color='darkred')
ax_plot_map(fig, ax2, opt, border=1, nsamples=200)
margin = 5e-2
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.add_patch(plt.Circle((0, 0), 1, color='limegreen', fill=False))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
ax2.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

###
#ax3.set_aspect('equal')
ax3.axis('equal')
ax3.plot(np.array((-4, 4)), np.array((0, 0)), linestyle='-.', color='black')
ax3.plot(np.array((0, 0)), np.array((-4, 4)), linestyle='-.', color='black')
ax_plot_track(opt, ax3, target=np.zeros(2),  aspect=True, track_color='darkred')
ax_plot_map(fig, ax3, opt, border=1, nsamples=200)
ax3.set(xlim=(-.01, .18), ylim=(-1e-3, 1e-3))
ax3.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax3.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))


###
#ax4.set_aspect('equal')
ax4.axis('equal')
ax4.plot(np.array((-4, 4)), np.array((0, 0)), linestyle='-.', color='black')
ax4.plot(np.array((0, 0)), np.array((-4, 4)), linestyle='-.', color='black')
ax_plot_track(opt, ax4, target=np.zeros(2),  aspect=True, track_color='darkred')
ax_plot_map(fig, ax4, opt, border=1, nsamples=200)
ax4.set(xlim=(.083, .086), ylim=(-1e-3, 1e-3))
ax4.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax4.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))


###
ax1.set_xlabel(f'a)')
ax2.set_xlabel(f'b)')
ax3.set_xlabel(f'c)')
ax4.set_xlabel(f'd)')
plt.savefig('mem/rsc/ex1-9.png', dpi=120, bbox_inches='tight')

np.linalg.norm(opt.optimal_vertex)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.08365341470192740388

In [94]:
start_x = np.array((0.6052, -0.9224))  # radius = 2
opt = pyramid_faces_optimizer(start_x=start_x, faces=4, rhobeg=.1)
for _ in opt.g_run(): pass

plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(10, 9))
ax1 = axes[0,0]
ax2 = axes[0,1]
ax3 = axes[1,0]
ax4 = axes[1,1]

circle = plt.Circle((0, 0), 1, color='limegreen', fill=False)

###
#ax1.set_aspect('equal')
ax1.axis('equal')

ax1.plot(np.array((-4, 4)), np.array((0, 0)), linestyle='-.', color='black')
ax1.plot(np.array((0, 0)), np.array((-4, 4)), linestyle='-.', color='black')

ax_plot_track(opt, ax1, target=np.zeros(2),  aspect=True, track_color='darkred')
ax_plot_map(fig, ax1, opt, border=2, nsamples=200)
ax1.add_patch(circle)
ax1.set(xlim=(-1, 1), ylim=(-1, 1))
ax1.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax1.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

###
#ax2.set_aspect('equal')
ax2.axis('equal')
ax2.plot(np.array((-4, 4)), np.array((0, 0)), linestyle='-.', color='black')
ax2.plot(np.array((0, 0)), np.array((-4, 4)), linestyle='-.', color='black')

ax_plot_track(opt, ax2, target=np.zeros(2),  aspect=True, track_color='darkred')
ax_plot_map(fig, ax2, opt, border=1, nsamples=200)
margin = 5e-2
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.add_patch(plt.Circle((0, 0), 1, color='limegreen', fill=False))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
ax2.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

###
#ax3.set_aspect('equal')
ax3.axis('equal')
ax3.plot(np.array((-4, 4)), np.array((0, 0)), linestyle='-.', color='black')
ax3.plot(np.array((0, 0)), np.array((-4, 4)), linestyle='-.', color='black')
ax_plot_track(opt, ax3, target=np.zeros(2),  aspect=True, track_color='darkred')
ax_plot_map(fig, ax3, opt, border=1, nsamples=200)
ax3.set(xlim=(-.5, .5), ylim=(-.5, .5))
ax3.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax3.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))


###
#ax4.set_aspect('equal')
ax4.axis('equal')
ax4.plot(np.array((-4, 4)), np.array((0, 0)), linestyle='-.', color='black')
ax4.plot(np.array((0, 0)), np.array((-4, 4)), linestyle='-.', color='black')
ax_plot_track(opt, ax4, target=np.zeros(2),  aspect=True, track_color='darkred')
ax_plot_map(fig, ax4, opt, border=1, nsamples=200)
ax4.set(xlim=(-1.5e-11, 1.5e-11), ylim=(-1.5e-11, 1.5e-11))
#ax4.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
#ax4.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))


###
ax1.set_xlabel(f'a)')
ax2.set_xlabel(f'b)')
ax3.set_xlabel(f'c)')
ax4.set_xlabel(f'd)')
plt.savefig('mem/rsc/ex1-10.png', dpi=120, bbox_inches='tight')

np.linalg.norm(opt.optimal_vertex)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.0024370510201885988e-12

# Two peaks

In [95]:
def gaussian_multipeak_optimizer(g1, g2, start_x, border=1, rhobeg=.1, rhoend=1e-12):
    mu1, sig1, A1 = g1
    mu2, sig2, A2 = g2
    G1 = functools.partial(tc.gaussian, mu=mu1, sig=sig1, A=A1)
    G2 = functools.partial(tc.gaussian, mu=mu2, sig=sig2, A=A2)
    F = lambda x: G1(x) + G2(x)
    
    c1 = lambda x: border - x[0]
    c2 = lambda x: border + x[0]
    c3 = lambda x: border - x[1]
    c4 = lambda x: border + x[1]
    C = (c1, c2, c3, c4)
    
    opt = GCobyla(start_x, F, C, rhobeg=rhobeg, rhoend=rhoend)
    return opt

In [96]:
import string

mu1 = (.5, .5)
sig1 = (.25, .25)
A1 = -2

mu2 = (-.5, -.5)
sig2 = (.25, .25)
A2 = -1

g1 = (mu1, sig1, A1)
g2 = (mu2, sig2, A2)


opts = (
    gaussian_multipeak_optimizer(g1, g2, start_x=(-.046, -.046)),
    gaussian_multipeak_optimizer(g1, g2, start_x=(-.5, .5)),
    gaussian_multipeak_optimizer(g1, g2, start_x=(.5, -.5)),
    gaussian_multipeak_optimizer(g1, g2, start_x=(-.047, -.047)),
    gaussian_multipeak_optimizer(g1, g2, start_x=(-.5, .3)),
    gaussian_multipeak_optimizer(g1, g2, start_x=(.3, -.5))
)

plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 9))
fig.tight_layout(pad=2.0)

for opt, ax, label in zip(opts, axes.ravel(), string.ascii_lowercase):
    for _ in opt.g_run(): pass

    ax_plot_map(fig, ax, opt, border=1.2, nsamples=200)
    ax_plot_track(opt, ax, target=opt.optimal_vertex, aspect=True, track_color='darkred')

    region = np.array(((1, 1), (-1, 1), (-1, -1), (1, -1), (1, 1)))
    ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='limegreen')
    ax.plot(np.array((-2, 2)), np.array((0, 0)), linestyle='-.', color='royalblue')
    ax.plot(np.array((0, 0)), np.array((-2, 2)), linestyle='-.', color='royalblue')
    ax.set(xlim=(-1.1, 1.1), ylim=(-1.1, 1.1))
    ax.set_xlabel(f'{label}) start x at {opt.start_x}')
    
plt.savefig('mem/rsc/ex1-11.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [97]:
n_points = 1000  # 2 x n_points dots
inner_border = .15
outer_border = 2

plt.close('all')
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax_plot_map(fig, ax, opt, border=outer_border, nsamples=300)

region = np.array(((1, 1), (-1, 1), (-1, -1), (1, -1), (1, 1)))
ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='limegreen')
ax.plot(np.array((-2, 2)), np.array((0, 0)), linestyle='-.', color='royalblue')
ax.plot(np.array((0, 0)), np.array((-2, 2)), linestyle='-.', color='royalblue')
ax.set(xlim=(-outer_border, outer_border), ylim=(-outer_border, outer_border))

color_1, peak_1 = 'red', np.array((.5, .5)) 
color_2, peak_2 = 'deepskyblue', np.array((-.5, -.5))
tol = 1e-5

ax.plot(peak_1[0], peak_1[1], linestyle='', marker='*', color=color_1)
ax.plot(peak_2[0], peak_2[1], linestyle='', marker='*', color=color_2)

targets = {color_1: [], color_2: []}
for _ in range(n_points):
    x_up = np.random.uniform(low=-inner_border, high=outer_border)
    y_down = np.random.uniform(low=-outer_border, high=inner_border)
    start_x = np.array((x_up, y_down)) 
    opt = gaussian_multipeak_optimizer(g1, g2, start_x=start_x)
    opt.run()
    
    color, peak = (color_1, peak_1) if (opt.optimal_vertex[0] > 0) else (color_2, peak_2)
    assert np.linalg.norm(opt.optimal_vertex - peak) < 1e-6
    targets[color].append(start_x) 
    
    x_down = np.random.uniform(low=-outer_border, high=inner_border)
    y_up = np.random.uniform(low=-inner_border, high=outer_border)
    start_x = np.array((x_down, y_up))
    opt = gaussian_multipeak_optimizer(g1, g2, start_x=start_x)
    opt.run()
    
    color, peak = (color_1, peak_1) if (opt.optimal_vertex[0] > 0) else (color_2, peak_2)
    assert np.linalg.norm(opt.optimal_vertex - peak) < tol
    targets[color].append(start_x)

for color, points in targets.items():
    points = np.array(points)
    ax.plot(points[:, 0], points[:, 1], linestyle='', marker='.', color=color)
    
plt.savefig('mem/rsc/ex1-12.png', dpi=120, bbox_inches='tight')  

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Waves field

In [98]:
import numpy as np

import tests.test_custom as tc


def wave_optimizer(T0, A0, T1, A1, start_x, border=1, rhobeg=1, rhoend=1e-12):
    F = lambda x: tc.waves(x, T0=T0, A0=A0, T1=T1, A1=A1)
    
    c1 = lambda x: border - x[0]
    c2 = lambda x: border + x[0]
    c3 = lambda x: border - x[1]
    c4 = lambda x: border + x[1]
    C = (c1, c2, c3, c4)
    
    opt = GCobyla(start_x, F, C, rhobeg=rhobeg, rhoend=rhoend)
    return opt

In [118]:
plt.close('all')
fig, ax = plt.subplots(1, 1, figsize=(6,6))

start_x = np.array((0,0))
T0, A0 = 1, 1
T1, A1 = 1, 1

rhobeg1 = .1
rhobeg2 = .8
rhobeg3 = 1

opt1 = wave_optimizer(T0=T0, A0=A0, T1=T1, A1=A1, start_x=start_x, border=1, rhobeg=rhobeg1)
opt2 = wave_optimizer(T0=T0, A0=A0, T1=T1, A1=A1, start_x=start_x, border=1, rhobeg=rhobeg2)
opt3 = wave_optimizer(T0=T0, A0=A0, T1=T1, A1=A1, start_x=start_x, border=1, rhobeg=rhobeg3)

for _ in opt1.g_run(): pass
for _ in opt2.g_run(): pass
for _ in opt3.g_run(): pass

ax_plot_map(fig, ax, opt1, border=1.2, nsamples=200)

ax_plot_track(opt1, ax, target=(.5, .5), aspect=True, track_color='red')
ax_plot_track(opt2, ax, target=(-.5, .5), aspect=True, track_color='blue')
ax_plot_track(opt3, ax, target=(.5, -.5), aspect=True, track_color='green')
region = np.array(((1, 1), (-1, 1), (-1, -1), (1, -1), (1, 1)))
ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='limegreen')

plt.savefig('mem/rsc/ex1-13.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [115]:
import string
import tests.data

plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(10, 9))
fig.tight_layout(pad=2.0)


opts = (
    wave_optimizer(T0=T0, A0=A0, T1=T1, A1=A1, start_x=tests.data.BAD_WAVES_FIELD_START_X[6], border=1, rhobeg=.1),
    wave_optimizer(T0=T0, A0=A0, T1=T1, A1=A1, start_x=tests.data.BAD_WAVES_FIELD_START_X[15], border=1, rhobeg=.1),
    wave_optimizer(T0=T0, A0=A0, T1=T1, A1=A1, start_x=tests.data.BAD_WAVES_FIELD_START_X[3], border=1, rhobeg=.1),
    wave_optimizer(T0=T0, A0=A0, T1=T1, A1=A1, start_x=tests.data.BAD_WAVES_FIELD_START_X[10], border=1, rhobeg=.1)
)

targets = ((-.5, .5), (.5, .5), (.5, .5), (.5, -.5))

for opt, ax, label, target in zip(opts, axes.ravel(), string.ascii_lowercase, targets):
    try:
        for _ in opt.g_run(): pass
    except:
        opt.optimal_vertex = opt.x
        
    ax_plot_map(fig, ax, opt, border=1.2, nsamples=200)
    ax_plot_track(opt, ax, target=target, aspect=True, track_color='deepskyblue')
    region = np.array(((1, 1), (-1, 1), (-1, -1), (1, -1), (1, 1)))
    ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='limegreen')
    ax.set_xlabel(f'{label}) start x at {opt.start_x}')

plt.savefig('mem/rsc/ex1-14.png', dpi=120, bbox_inches='tight') 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

maximum number of function evaluations reach
maximum number of function evaluations reach
maximum number of function evaluations reach
maximum number of function evaluations reach


In [28]:
plt.close('all')
fig, ax = plt.subplots(1,1, figsize=(6,6))

ax_plot_map(fig, ax, opt, border=1.2, nsamples=500)
region = np.array(((1, 1), (-1, 1), (-1, -1), (1, -1), (1, 1)))
ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='limegreen')
#ax.fill(region[..., 0], region[..., 1], color='limegreen', alpha=.15)

targets = tests.data.BAD_WAVES_FIELD_START_X
ax.plot(targets[:, 0], targets[:, 1], linestyle='', marker='.', color='orange')
plt.savefig('mem/rsc/ex1-15.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
ltests.data.BAD_WAVES_FIELD_START_X

array([[ 0.98339995,  0.39446042],
       [ 0.98339995,  0.39446042],
       [ 0.95591205, -0.36856458],
       [ 0.97530021,  0.7642675 ],
       [ 0.83423063,  0.95234139],
       [-0.8743952 ,  0.96539099],
       [-0.79177057,  0.95053853],
       [ 0.96794841,  0.78593216],
       [ 0.96696354, -0.41931914],
       [-0.31321551,  0.9760916 ],
       [ 0.97790963, -0.26520498],
       [ 0.97440277, -0.76728859],
       [ 0.96886712, -0.38365157],
       [ 0.95774139, -0.36911808],
       [ 0.78332253,  0.95034937],
       [ 0.99692965,  0.57384217],
       [ 0.69290577,  0.97301846],
       [ 0.10505047,  0.95227693],
       [ 0.58679939,  0.95906591],
       [ 0.33200481,  0.97066115],
       [ 0.98295483, -0.6934984 ],
       [-0.65645219,  0.98212985],
       [ 0.96140436,  0.57559995],
       [ 0.96863019,  0.22746172],
       [ 0.97787301, -0.7597252 ],
       [ 0.53560804,  0.99344428],
       [-0.7817761 ,  0.95530841],
       [-0.3477351 ,  0.98330356],
       [ 0.50306679,